# It could be cake <div style="text-align: right; float: right"> IANNWTF - Final Task </div>

##### The usual imports

In [1]:
import numpy as np
import os, time
import tensorflow as tf
import json

##### Recipe data script
It connects to a local MongoDB instance, that should be configured in-tune with the `config.json` file.

It then can build dictionaries for id2value mappings and batch generation

In [22]:
# %load recipes.py
import pymongo, json, numpy as np
from collections import Counter
from nltk.tokenize import RegexpTokenizer


class Recipes:
    """A helper class to interface with the dataset"""
    def __init__(
      self,
      useMongo=True,
      config="config.json"
    ):
        """set up connection to db"""

        with open(config) as cfg:
            self.data = json.load(cfg)
            storage = self.data['storage']
            CRAWL_FOLDER = storage['folder']
            RECIPE_COLLECTION = storage['collection']

        client = pymongo.MongoClient('mongodb://localhost:27017')
        self.db = client['iannwtf']
        self.coll = self.db[RECIPE_COLLECTION]

    def create_dictionaries(self,vocabulary_size=20000):
        """
        Counts all unique ingredients and creates a dictionary mapping.
        
        We aggregate the data in the database to get one object per ingredient,
        each containing its foodId and name. The ingredients should be already remapped
        into the interval [1, #ingredients].
        We then build two dictionary: ingredient name to id and vice versa
        There are also empty dictionaries for word2id and unit2id, since it was created
        with expansion in mind.
        
        Returns:
        --------
        lengths: tuple
            the length of each dictionary mapping
        """
        agg = self.coll.aggregate([
            {"$project": {"_id": 0, "ingredients.foodId": 1, "ingredients.name": 1}},
            {"$unwind": "$ingredients"},
            {
                "$group": {
                    "_id": "$ingredients.foodId",
                    "name": {"$first": "$ingredients.name"}
                }
            }
          ], allowDiskUse=True)

        
        # mapping names to ids. Dictionary comprehensions be praised.
        self._ing2id = {item['name'].lower(): int(item['_id']) for item in agg}
        # For whatever hecking reason, the chefkoch database has two entries for 'miracel whip'
        # they only differ in the capitalisation of ONE letter... but since the dictionary uses
        # lower case names, this broke the code
        # Since this was noticed fairly late into development, a more or less well suited fix was implemented
        self._ing2id['miracel whipmyass'] = 1362
        self._ing2id['NO_INGREDIENT'] = 0
        # and in reverse, for that super speed performance
        self._id2ing = dict(zip(self._ing2id.values(), self._ing2id.keys()))

        self._word2id = {}
        self._id2word = {} 
        self._unit2id = {}
        self._id2unit = {}

        ingredient_size = len(self._ing2id)
        unit_size = len(self._unit2id)

        return ingredient_size, unit_size, vocabulary_size

    def ings2ids(self, ings):
        """returns ids for a list of ingredient names"""
        return self._2(ings, self._ing2id, 0)
  
    def units2ids(self, units):
        """returns ids for a list of unit names"""
        return self._2(units, self._unit2id, 0)

    def words2ids(self, words):
        """returns ids for a list of words"""
        return self._2(words, self._word2id, 0)

    def ids2ings(self, ids):
        """returns ingredient names for a list of ids"""
        return self._2(ids, self._id2ing, "NO_INGREDIENT")
  
    def ids2units(self, ids):
        """returns unit names for a list of ids"""
        return self._2(ids, self._id2unit, "NO_UNIT")

    def ids2words(self, ids):
        """returns words for a list of ids"""
        return self._2(ids, self._id2word, "UNKNOWN")

    def _2(self, names, d, undef):
        """Extracts the values for the keys `names` out of dictionary `d` and uses `undef` if not found"""
        if type(names) in [list, range]:
            return [d.get(n, undef) for n in names]
        elif type(names) == np.ndarray:
            res = np.full_like(names, "", dtype=object)
            for i in range(res.shape[0]):
                res[i] = self._2(names[i], d, undef)
            return res
        else:
            return d.get(names, undef)

    def get_ingredient_batch(self, batchsize=25, max_ingredients=15):
        """Generate a batch of original recipes and pad or cut to length `max_ingredients`"""
        count = 0
        batch = np.zeros((batchsize, max_ingredients), dtype=np.float32)
        cursor = self.coll.find({}, {"ingredients.foodId": 1, "_id": 0}, no_cursor_timeout = True)

        recipes = np.array(list(cursor))
        np.random.shuffle(recipes)
        
        for doc in recipes:
            # ids could be strings. not cool.
            _ids = [int(x['foodId']) for x in doc['ingredients']]
            # cut off if necessary
            if len(_ids) > max_ingredients:
                _ids = _ids[:max_ingredients]
            batch[count][:len(_ids)] = _ids[:]
            count += 1

            if count == batchsize:
                # the batch is ready to be served
                yield batch
                count = 0
                batch = np.zeros((batchsize, max_ingredients), dtype=np.float32)

##### Helper functions
They just make the code that much more readable

In [16]:
def feed_forward_layer(x, target_size, activation_function = None):
    """
    Courtesy of Lukas Braun. Creates a feed forward layer from input x onto size target size.
    
    Parameters:
    -----------
    x: tensor
        the input tensor to the layer
    target_size: int
        on how many neurons to map the input
    activate_function: tensorflow activation function
        which activation function to use, if None returns non-activated drive
        
    Returns:
    --------
    activation: tensor
        the result of the feed forward layer, with activation function applied if supplied
    """
    print("Forward-Layer:" + str(x.shape))
    
    fan_in = int(x.shape[-1])
    
    if activation_function == tf.nn.relu:
        var_init = tf.random_normal_initializer(stddev = 2/fan_in)
    else:
        var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
    weights = tf.get_variable("weights", [x.shape[1], target_size], tf.float32, var_init)
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [target_size], tf.float32, var_init)
    
    activation = tf.matmul(x, weights) + biases
    
    return activation_function(activation) if callable(activation_function) else activation

def get_ings(embeddings, ingredients):
    '''
    This is used to find the ingredient id to the generated embedding of the lstm.
    We use the cosine similarity to find the closest embedding.
    
    Parameters:
    -----------
    embeddings: 2D Tensor
        the weight matrix with all ingredients representations shape: [ingredient_size, embedding_size]
    ingredients: 3D Tensor
        a matrix of ingedient vectors of who we want to find the nearest neigbhours
        
    Returns:
    --------
    closest_ings: 2D Tensor
        shape [batch_size, max_ingredients]. For each recipe in the batch, it now holds the ingredient ids
      
    '''''
    # we need to normalise for cosine similarity to work
    normed_embedding = tf.nn.l2_normalize(embeddings, dim=1)   

    # unstacking and restacking since matrix multiplications only work on the same rank
    closest_ings = []
    for batch in tf.unstack(ingredients, axis=1):
        batch = tf.nn.l2_normalize(batch, dim=1)
        closest_ings.append(tf.argmax(tf.matmul(batch, tf.transpose(normed_embedding, [1, 0])), axis=1))

    closest_ings = tf.stack(closest_ings, axis=1)
    
    return closest_ings

def discriminator_convolution(x, window_size) : 
    """
    Convolutes over the recipe in its embedding form, to extract features.
    Which are used to discriminate between the original recipes and the generated ones.
    
    Params:
    -------
    x: 3D Tensor
        The tensor holding the original recipes and the generated ones with their embeddings
    window_size:  int
        the window size to use in the convolution
        
    Returns:
    --------
    convolution: 2d tensor
        the result of max pooling after the convoliution for feature extraction
    """
    
    x = tf.reshape(x,[2*batch_size, embedding_size, max_ingredients, 1])
    fan_in = 1
    for dim in x.shape[1:]: fan_in * dim.value
    
    var_init = tf.random_normal_initializer(stddev=fan_in**1/2)
    kernel = tf.get_variable("kernel", [embedding_size, window_size, 1,1], tf.float32, var_init)
    
    y = tf.nn.conv2d(x, kernel, strides = [1, 1, 1, 1], padding = "VALID")    
    new_length = y.get_shape()[2]
    
    var_init = tf.constant_initializer(0.0)
    biases = tf.get_variable("biases", [1,1, new_length, 1], initializer = var_init)
    
    y = y + biases
    
    return tf.squeeze(tf.nn.max_pool(y, [1,1, new_length, 1], strides = [1, 1, 1, 1], padding = "VALID"))

def generator_similarity_loss(batch):
    """
    This loss is introduced to make the network choose more diverse ingredients, instead of repeating
    ingredients and using close ingredients
    
    Params:
    -------
    batch: 3d Tensor
        The generated recipe batch
        
    Returns:
    --------
        The mean cosine similarity for each recipe to itself
    """
    def iterate(recipe):
        tn = tf.nn.l2_normalize(recipe, dim=1)
        similarity = tf.matmul(tn, tf.transpose(tn, [1,0]))
        sim = tf.reduce_mean(similarity)
        return sim
    
    all_sims = tf.map_fn(iterate, batch)
        
    
    return tf.stack(all_sims)

##### Loading recipe data, defining hyperparameters
Load values from the config file and setup hyper parameters

In [17]:
config = "config.json"
recipes = Recipes(config)

with open(config,"r",encoding="utf-8") as cfg:
    settings = json.load(cfg)["network"]

hyperparams = settings["hyperparameters"]
directories = settings["directories"]

epochs = 12
gen_lr = 0.005
dis_lr = 0.001
beta1 = 0.5
batch_size = 50
lambda_ing = 0.4 # coefficient to use for the custom loss

# the pretrained weights
embedding_weights = directories["embedding_weights"]

# for tensorboard
weight_dir = directories["weight_dir"]
summary_dir = directories["summary_dir"]
     
## Hyperparameters
embedding_size = hyperparams["embedding_size"]
# size of the random input vector
z_size = hyperparams["z_size"]
max_ingredients = hyperparams["max_ingredients"]
memory_size = embedding_size
dropout_rate = hyperparams["dropout_rate"]
optimizer = tf.train.AdamOptimizer

# and create the dictionaries to look up the ingredients.
# unit_size & vocab size are unused and were implemented for further extension of the network
ingredient_size, unit_size, vocabulary_size = recipes.create_dictionaries()

print("""embedding size : {},
input size : {},
max ingredients : {},
memory size : {},
dropoutrate : {},
ingredient size : {}"""\
      .format(embedding_size, z_size, max_ingredients, memory_size, dropout_rate, ingredient_size))

embedding size : 64,
input size : 50,
max ingredients : 10,
memory size : 64,
dropoutrate : 0.85,
ingredient size : 3601


##### Defining the graph
For more details on the architecture, see the supplied pdf file

In [18]:
# Not resetting causes conflicts in jupyter as the previous executed graph is still active in memory
tf.reset_default_graph()


###### The generator network. ######      
with tf.variable_scope("generator"):

    ##### placeholders  #####
    # for input vector, and lstm state
    with tf.variable_scope("inputs"):
        input_vec = tf.placeholder(tf.float32,[batch_size,z_size],name="input_vec")
        
        cell_state = tf.placeholder(tf.float32,[batch_size, memory_size],name="cell_state")
        hidden_state = tf.placeholder(tf.float32,[batch_size, memory_size],name="hidden_state")
    
    
    ##### embedding #####
    # it is not trainable as it was pre trained to a sufficient degree
    with tf.variable_scope("embedding"):
        init = tf.random_uniform_initializer(-1.0,1.0)
        embeddings = tf.get_variable("embed",
                                     [ingredient_size, embedding_size],
                                     initializer=init,
                                     trainable=False
                                    )
  
        
    ##### RNN #####
    # nothing special about it. We repeatedly feed the input vector for each ingredient to be produced
    with tf.variable_scope("rnn"):
        cell = tf.nn.rnn_cell.BasicLSTMCell(memory_size)
        cell= tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout_rate)
        zero_state = cell.zero_state(batch_size, tf.float32)
             
        init_state = tf.nn.rnn_cell.LSTMStateTuple(c = cell_state, h = hidden_state)
        inputs = [input_vec for i in range(max_ingredients)]
        
        lstm_out, lstm_state = tf.nn.static_rnn(cell, inputs, initial_state=init_state)
        
        # has shape [max_ingredients, batch_size, embedding], but we need batch_size major
        outputs = tf.transpose(lstm_out, [1,0,2])

    # get trainable variables for generator
    train_gen = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)


##### The discriminator network #####
with tf.variable_scope("discriminator"):
    
    ##### placeholder for originals & batch creation #####
    with tf.variable_scope("input"):
        orig_recipes_placeholder = tf.placeholder(tf.int64,
                                                  [batch_size,max_ingredients],
                                                  name="orig_recipes_placeholder"
                                                 )
        orig_recipes = tf.nn.embedding_lookup(embeddings, orig_recipes_placeholder)

        # create input batch for discriminator
        batch = tf.concat([outputs, orig_recipes],axis=0)
    
    ##### CNN #####
    with tf.variable_scope("convolutions"):
        convs = []
        for i in range(2,max_ingredients+1):
            with tf.variable_scope("conv"+str(i)):
                conv = discriminator_convolution(batch, i)
                convs.append(conv)
        
        conv_outputs = tf.stack(convs, axis=1)
     
    ##### Single Node reduction #####
    with tf.variable_scope("readout"):
        logits = feed_forward_layer(conv_outputs,1,None)
                                    
    # get trainable variables for discriminator
    train_dis = tf.get_variable_scope().get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

##### Evaluation and training of the network #####
with tf.variable_scope("evaluation"):
    
    # to look at what glorious ingredient compilations it produced
    readable_outputs = get_ings(embeddings, outputs)
    
    # to compute cross entropy for generator.
    # consists of only "1" because this is the discriminator label for "real"
    gen_labels = tf.ones((batch_size,1))
    
    # only the first half of the output from the discriminator concerns the pictures
    # produced by the generator
    # so only get first half of logits (which equals the batch_size)
    gen_logits = logits[:batch_size]
    
    # for discriminator cross entropy.
    # first half of input are fake images ("0"), second half real ones ("1")
    dis_labels = tf.concat((tf.zeros((batch_size,1)),tf.ones((batch_size,1))),axis=0)
    dis_logits = logits

    # calculating the loss for generator and discriminator
    gen_loss_mult_all = generator_similarity_loss(outputs)
    gen_loss_multing = tf.reduce_mean(gen_loss_mult_all) * lambda_ing
    gen_loss_ce = tf.nn.sigmoid_cross_entropy_with_logits(labels=gen_labels, logits=gen_logits)
    gen_loss = gen_loss_ce + gen_loss_multing
    
    dis_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=dis_labels, logits=dis_logits)
    
    # initialize optimizer
    gen_optimizer = tf.train.AdamOptimizer(learning_rate=gen_lr, beta1=beta1)
    dis_optimizer = tf.train.AdamOptimizer(learning_rate=dis_lr, beta1=beta1)
    
    # define training steps with respective variable lists
    global_step = tf.get_variable("global_step",[],tf.int32,trainable=False)
    gen_step = gen_optimizer.minimize(gen_loss, var_list=train_gen, global_step=global_step)
    dis_step = dis_optimizer.minimize(dis_loss, var_list=train_dis, global_step=global_step)
    
    # tensorboard things
    tf.summary.scalar("generator_loss", tf.reduce_mean(gen_loss))
    tf.summary.scalar("discriminator_loss", tf.reduce_mean(dis_loss))
    summaries = tf.summary.merge_all()

Forward-Layer:(100, 9)


##### This is where we train our network
Welcome to the gym!

In [19]:
with tf.Session() as session:
    
    ## initialising directories if they do not exist
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    if not os.path.exists(summary_dir):
        os.makedirs(summary_dir)
        
    # To save what we do. Like a diary, but for ann's
    train_saver = tf.train.Saver()
    train_writer = tf.summary.FileWriter(summary_dir, session.graph)
    
    ## Try to load trained data and initialise variables if there is none
    ckpt = tf.train.latest_checkpoint(weight_dir)
    if ckpt is None:
        print("No checkpoint found. Initialising variables.")
        session.run(tf.global_variables_initializer())
    else:
        train_saver.restore(session, ckpt)    
    
    # load the pre trained embedding. If it does not exist, this is not gonna work.
    embed_saver = tf.train.Saver(var_list={'ingredient_embedding': embeddings})
    if os.path.isdir(embedding_weights):
        embed_ckpt = tf.train.latest_checkpoint(embedding_weights)
        if embed_ckpt is not None:
            embed_saver.restore(session, embed_ckpt)
        else:
            raise Exception("Embedding weights not found")
    else:
        raise Exception("Embedding weights folder not found")
    
    
    ## Train all the epochs and save the summaries for each timestep
    total_time = 0
    for epoch in range(1, epochs+1):
        
        print("Starting epoch %d..." %epoch)
        
        # get batch of original recipes #mnist.get_batch(batch_size)
        real_data = recipes.get_ingredient_batch(batch_size, max_ingredients)
        t = time.time()
        
        counter = 0
        
        for data in real_data:
            # lstm initial state will be 0
            _state = session.run(zero_state)
            
            ## for each batch we need a new set of random vectors
            z_vec = np.random.uniform(-1, 1, (batch_size,z_size))            
            
            # feed data into placeholders
            _feed_dict = {input_vec: z_vec, 
                         orig_recipes_placeholder: data,
                         cell_state: _state.c,
                         hidden_state: _state.h,}
            
            # let's train our network!
            _dStep, _gStep, _summ, _step, _fr = session.run([
                                                    dis_step, gen_step,
                                                    summaries, global_step,
                                                    readable_outputs
                                                ],
                                                feed_dict=_feed_dict)
            
            train_writer.add_summary(_summ, _step)
            
            # create the ingredient names. numbers are hard to read
            _fr = recipes.ids2ings(_fr)
            # output every 100 steps to look at the nice graphs            
            if counter % 100 == 0 :
                for rec in np.random.choice(range(batch_size),5):
                    print("|| ",end='')
                    for i in _fr[rec]:
                        print(i," + ",end='')
                    print(" ||")
                print("------------------------------------------")
            
            counter = counter + 1
            
        t = time.time() - t
        total_time += t
        est = t * (epochs - epoch)
        
        ## This is just the time formatting. Move along.
        m, s = divmod(t, 60)
        e_m, e_s = divmod(est, 60)
        
        print(("Finished epoch in {0: .0f}m{1: .2f}s. "+\
              "Estimated remaining time: {2: .0f}m{3: .2f}s").format(m,s,e_m,e_s))
        
        # and save the weights
        train_saver.save(session, weight_dir, global_step=_step)
    
    m, s = divmod(total_time, 60)
    print("Took a total of {0: .0f}m{1: .2f}s".format(m,s))

No checkpoint found. Initialising variables.
INFO:tensorflow:Restoring parameters from train_data/embedding_weights/model.ckpt
Starting epoch 1...
|| butter  + butter  + wasser  + wasser  + butter  + wasser  + wasser  + butter  + butter  + wasser  +  ||
|| mehl  + ei(er)  + ei(er)  + ei(er)  + gewürzgurke(n)  + ei(er)  + ei(er)  + ei(er)  + mehl  + zucker  +  ||
|| sahne  + sojasauce  + sahne  + knoblauch  + knoblauch  + knoblauch  + sahne  + chilischote(n)  + chilischote(n)  + knoblauch  +  ||
|| apfel  + banane(n)  + apfel  + apfel  + apfel  + apfel  + banane(n)  + apfel  + apfel  + apfel  +  ||
|| wodka  + ananas  + zwiebel(n)  + ananas  + porree  + ananas  + ananas  + ananas  + ananas  + ananas  +  ||
------------------------------------------
|| basilikum  + basilikum  + basilikum  + mehl  + ei(er)  + zitrone(n)  + zwiebel(n)  + wasser  + gewürznelke(n)  + wasser  +  ||
|| basilikum  + basilikum  + basilikum  + mehl  + ei(er)  + zitrone(n)  + zwiebel(n)  + wasser  + gewürznelke(n)

|| schlagsahne  + joghurt  + apfel  + essig  + zwiebel(n)  + sojasauce  + sahne  + mehl  + mehl  + zucker  +  ||
|| ricotta  + joghurt  + honig  + kartoffel(n)  + zwiebel(n)  + tomate(n)  + zitronensaft  + mehl  + zucker  + speisestärke  +  ||
|| schlagsahne  + honig  + honig  + salz  + zwiebel(n)  + tomate(n)  + sahne  + butter  + zucker  + zucker  +  ||
|| sahne  + kürbiskerne  + honig  + essig  + zwiebel(n)  + tomate(n)  + zitronensaft  + mehl  + wasser  + butter  +  ||
|| schlagsahne  + kürbiskerne  + honig  + salz  + zwiebel(n)  + tomate(n)  + zitronensaft  + butter  + zucker  + butter  +  ||
------------------------------------------
|| sahne  + joghurt  + honig  + apfel  + zwiebel(n)  + tomate(n)  + zitronensaft  + zucker  + limette(n)  + wasser  +  ||
|| skyr  + joghurt  + honig  + kartoffel(n)  + zwiebel(n)  + tomate(n)  + zitrone(n)  + limette(n)  + wasser  + milch  +  ||
|| sahne  + joghurt  + haferflocken  + salz  + zwiebel(n)  + tomate(n)  + zitrone(n)  + wasser  + wasser 

|| schlagsahne  + joghurt  + apfel  + kartoffel(n)  + gemüsebrühe  + tomate(n)  + zitrone(n)  + sahne  + wasser  + butter  +  ||
|| sahne  + joghurt  + apfel  + kartoffel(n)  + salz  + sahne  + lasagneplatte(n)  + sahne  + wasser  + butter  +  ||
|| sahne  + joghurt  + apfel  + kartoffel(n)  + gemüsebrühe  + tomate(n)  + zitrone(n)  + sahne  + wasser  + butter  +  ||
|| sahne  + joghurt  + haferflocken  + salz  + gemüsebrühe  + tomate(n)  + ei(er)  + sahne  + wasser  + butter  +  ||
|| sahne  + joghurt  + apfel  + kartoffel(n)  + gemüsebrühe  + tomate(n)  + milch  + sahne  + wasser  + butter  +  ||
------------------------------------------
|| schlagsahne  + joghurt  + apfel  + salz  + gemüsebrühe  + tomate(n)  + zitrone(n)  + eis  + wasser  + wasser  +  ||
|| schlagsahne  + joghurt  + apfel  + salz  + gemüsebrühe  + tomate(n)  + zitrone(n)  + eis  + wasser  + wasser  +  ||
|| natron  + joghurt  + apfel  + salz  + gemüsebrühe  + tomate(n)  + zitrone(n)  + ananassaft  + zucker  + lorbee

|| schlagsahne  + sonnenblumenkerne  + zucchini  + zwiebel(n)  + knoblauch  + salz  + butter  + sahnesteif  + zucker  + zimt  +  ||
|| joghurt  + kürbiskerne  + amarant  + kreuzkümmel  + tomate(n)  + dill  + lasagneplatte(n)  + mascarpone  + zucker  + eis  +  ||
|| schlagsahne  + sonnenblumenkerne  + brokkoli  + zwiebel(n)  + tomate(n)  + dill  + puddingpulver  + mehl  + zucker  + ananassaft  +  ||
|| schlagsahne  + kümmel  + sesam  + zwiebel(n)  + tomate(n)  + salz  + lasagneplatte(n)  + mascarpone  + eis  + wodka  +  ||
|| natron  + schnittlauch  + sesam  + tomate(n)  + sojasauce  + zitronensaft  + butter  + zitrone(n)  + zucker  + apfelsaft  +  ||
------------------------------------------
|| schlagsahne  + sonnenblumenkerne  + blattspinat  + zucchini  + tomate(n)  + dill  + mehl  + zitronensaft  + zucker  + zimt  +  ||
|| joghurt  + salatgurke(n)  + sesam  + chilipaste  + sojasauce  + dill  + lasagneplatte(n)  + mehl  + zucker  + zimt  +  ||
|| mandarine(n)  + sonnenblumenkerne  + 

|| reispapier  + schinken  + südwein  + schweineschnitzel  + knoblauchbutter  + gewürzgurke(n)  + madeira  + peperoncino  + spinat  + pinkelwürste  +  ||
|| entenbrüste  + lachs  + schwarzwälder schinken  + schweineschnitzel  + single malt whisky  + jodsalz  + freekeh (gerösteter grüner weizen)  + portionsgläser  + leber  + thai-currypulver  +  ||
|| farfalle  + tofu  + schwarzwälder schinken  + tomatenketchup  + einmachgewürz  + kirschlikör  + weichweizengrieß  + ruchmehl  + forellenfilet(s)  + spargelspitzen  +  ||
|| blattsalat  + tofu  + hähnchenunterkeule(n)  + schweineschnitzel  + sauerteig  + worcestersauce  + freekeh (gerösteter grüner weizen)  + butter, weiche  + weißwein  + speiseöl  +  ||
|| farfalle  + schinken  + pumpernickeltaler  + schweineschnitzel  + trockenhefe  + gewürzgurke(n)  + pul biber  + hornhecht(e)  + forellenfilet(s)  + lammfilet(s)  +  ||
------------------------------------------
|| dressing  + schinken  + papayakerne  + rauke  + feldsalat  + jodsalz  + pi

...

|| maggi  + speck  + estragon  + kräuter  + fondor  + erbsen  + butterschmalz  + kapern  + schinken  + senf  +  ||
|| worcestersauce  + cornichons  + liebstöckel  + kräuter  + pellkartoffeln  + pesto  + schinkenspeck  + gemüse  + gemüse  + weißkohl  +  ||
|| steinpilze  + pökelfleisch  + bärlauch  + walnüsse  + dinkelvollkornmehl  + carboxymethylcellulose  + bauchspeck  + waldpilzfond  + lachssteak(s)  + schinkenspeck  +  ||
|| majoran  + kerbel  + pellkartoffeln  + bärlauch  + fleischkäse  + semmel(n)  + schweinesteak(s)  + remoulade  + wurst  + fleischtomate(n)  +  ||
|| bacon  + kerbel  + kerbel  + bärlauch  + gehacktes  + kochschinken  + kochschinken  + bacon  + knoblauchpulver  + wurst  +  ||
------------------------------------------
|| bacon  + wurst  + chilipulver  + mais  + pilze  + fleisch  + tomatenketchup  + rinderbrühe  + bohnen  + brühe  +  ||
|| bacon  + gehacktes  + maiskörner  + maggi  + kochschinken  + muschel(n)  + bohnen  + gemüse  + schweineschnitzel  + bacon  +  |

|| bacon  + kabeljaufilet(s)  + pfefferschote(n)  + gehacktes  + spinat  + kartoffelpüreepulver  + schweinekotelett(s)  + bratwurst  + kräuter  + kalbsfond  +  ||
|| paprika  + sahne-schmelzkäse  + sahne-schmelzkäse  + gemüsezwiebel(n)  + karotte(n)  + gemüsezwiebel(n)  + krabben  + gemüsebrühwürfel  + gemüsezwiebel(n)  + gemüsezwiebel(n)  +  ||
|| sauerampfer  + runkelrübe(n)  + makrele(n)  + gänseklein  + hoden  + roggen  + schokotropfen  + fliederblütendolden  + pfannkuchen  + modellierschokolade  +  ||
|| gouda  + makrele(n)  + brottrunk  + hoden  + graubrot  + schimmelkäse  + steinpilze  + chilisalz  + seitanmehl  + jiaogulanblätter  +  ||
|| chilipulver  + kräuter  + kräuter  + sahne-schmelzkäse  + kirschtomate(n)  + kümmelpulver  + scampi  + muschel(n)  + bacon  + bandnudeln  +  ||
------------------------------------------
|| longanfrüchte  + schinkenspeck  + bratensaft  + rahmspinat  + bärlauch  + lammfilet(s)  + kartoffelpüreepulver  + malzbier  + kalbsschnitzel  + pfeffer, g

|| porree  + porree  + hähnchen  + putenbrust  + pilze  + meerrettich  + worcestersauce  + suppengrün  + worcestersauce  + saucenbinder  +  ||
|| camembert(s)  + spitzpaprika  + paprika  + matjesfilet(s)  + endiviensalat  + cheddarkäse  + bärlauch  + chicorée  + blauschimmelkäse  + bärlauch  +  ||
|| gouda  + harissa  + emmentaler  + kräuterbutter  + bärlauch  + bärlauch  + bärlauch  + stangenweißbrot(e)  + steviapulver  + chilisalz  +  ||
|| hokkaidokürbis(se)  + jodsalz  + gouda  + mangold  + spillinge  + weingummi  + cheddarkäse  + weizenschrot  + chayote(n)  + gnocchi-nudeln  +  ||
|| fisch  + hähnchenschnitzel  + blumenkohl  + brühe  + paprika  + kamille  + cheddarkäse  + rosenkohl  + pfifferlinge  + räucherlachs  +  ||
------------------------------------------
|| porree  + blumenkohl  + bohnen  + bohnen  + bohnen  + worcestersauce  + pilze  + bohnen  + kasseler  + schweinefilet(s)  +  ||
|| erbsen  + porree  + rinderfilet(s)  + erbsen  + garam masala  + bohnen  + pilze  + bohnen

|| gehacktes  + hüttenkäse  + bulgur  + rinderfilet(s)  + bacon  + harissa  + bärlauch  + kräuterbutter  + maltose  + brunnenkresse  +  ||
|| fleischtomate(n)  + kohlwurst  + cabanossi  + fisch  + weinblätter  + chili  + chiliflocken  + meerrettich  + zitronenpfeffer  + muschel(n)  +  ||
|| kräuterbutter  + krabben  + tintenfisch(e)  + bratwürste  + hähnchenbrüste  + hähnchenbrüste  + brunnenkresse  + kresse  + pfifferlinge  + nordseekrabben  +  ||
|| kräuterbutter  + krabben  + tintenfisch(e)  + bratwürste  + hähnchenbrüste  + hähnchenbrüste  + brunnenkresse  + kresse  + pfifferlinge  + nordseekrabben  +  ||
|| bandnudeln  + blumenkohl  + muschel(n)  + wurst  + fleisch  + blumenkohl  + weißbrot  + harissa  + schweinenacken  + rinderhackfleisch  +  ||
------------------------------------------
|| gemüse  + rinderbrühe  + fondor  + kräuter  + gemüse  + ketchup  + bacon  + bacon  + pilze  + bacon  +  ||
|| urkohl  + weintrauben  + klebreismehl  + brennnesselsamen  + landjäger  + dinkelvo

|| tonkabohne(n)  + chiasamen  + getreide  + dinkelvollkornmehl  + schokotropfen  + birnenhälfte(n)  + johannisbrotkernmehl  + bismarckhering(e)  + sauermilch  + samen  +  ||
|| schinkenspeck  + muschel(n)  + bärlauch  + mangold  + worcestershiresauce  + gemüsebrühwürfel  + rinderbrühe  + scampi  + bacon  + muschel(n)  +  ||
|| paprikawurst  + rinderfilet(s)  + kräuterbutter  + tomatenpüree  + gemüsebrühwürfel  + pfifferlinge  + frühstücksspeck  + fuchs vanillezucker  + tomatenpüree  + weißbrot  +  ||
|| wirsing  + kohlrabi  + gemüsezwiebel(n)  + krabben  + blumenkohl  + gemüsezwiebel(n)  + erbsen  + spinat  + bohnen  + liebstöckel  +  ||
|| gemüsefond  + wirsing  + kürbiskernöl  + wurst  + maggi  + schinkenspeck  + worcestershiresauce  + pfifferlinge  + rinderfilet(s)  + rinderbrühe  +  ||
------------------------------------------
|| frühstücksspeck  + zwiebelpulver  + meerrettich  + rosenkohl  + frühstücksspeck  + bandnudeln  + blauschimmelkäse  + kräutersalz  + bacon  + spitzpaprik